In [32]:
rm(list = ls())

In [33]:
# load necessary packages
pacman::p_load(foreign,
                data.table,
                tidyverse,
                survival,
                survivalAnalysis,
                muhaz,
               ggfortify,
               ranger,
               coin,
               survminer,
               GGally
              )

In [55]:
library(fastDummies)

In [118]:
load("pos_and_historic2.RData")

In [35]:
#pos_and_historic2[,profit := as.integer(profit)]
#pos_and_historic2[,loss := as.integer(loss)]

In [36]:
surv_cp1 <- Surv(time = pos_and_historic2$days_start,
                time2 = pos_and_historic2$days,
               event = pos_and_historic2$status)

In [37]:
cox_cp2 <- coxph(surv_cp1 ~ profit, id = pos_no2, cluster = pos_no2, data = pos_and_historic2)

In [38]:
cox_cp2 %>% summary

Call:
coxph(formula = surv_cp1 ~ profit, data = pos_and_historic2, 
    id = pos_no2, cluster = pos_no2)

  n= 6573810, number of events= 147964 
   (27335 observations deleted due to missingness)

               coef exp(coef) se(coef) robust se     z Pr(>|z|)    
profitTRUE 0.620083  1.859082 0.005231  0.005550 111.7   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

           exp(coef) exp(-coef) lower .95 upper .95
profitTRUE     1.859     0.5379     1.839     1.879

Concordance= 0.575  (se = 0.001 )
Likelihood ratio test= 14109  on 1 df,   p=<2e-16
Wald test            = 12482  on 1 df,   p=<2e-16
Score (logrank) test = 14502  on 1 df,   p=<2e-16,   Robust = 11467  p=<2e-16

  (Note: the likelihood ratio and score tests assume independence of
     observations within a cluster, the Wald and robust score tests do not).

In [109]:
probs1 <- seq(0, 1, length.out = 11)
probs1

[1] 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0

In [110]:
breaks1 <- quantile(pos_and_historic2$overhang, probs = probs1, na.rm = T)
breaks1

0%           10%           20%           30%           40% 
-1.077715e+00 -2.328169e-01 -1.438346e-01 -8.937066e-02 -5.094425e-02 
          50%           60%           70%           80%           90% 
-2.241017e-02  1.163119e-05  2.728509e-02  7.585065e-02  1.702879e-01 
         100% 
 1.934585e+00

In [119]:
pos_and_historic2[, overhang_fac := cut(overhang,
                                       breaks = quantile(overhang, probs = probs1, na.rm = T),
                                       labels = probs1[-1])]

In [120]:
pos_and_historic2

date,value,MENKUL_KODU,KAPANIS,AOF,MIN,MAX,ISLEM_MIKTARI,ISLEM_HACMI,pos_no2,⋯,av_cost_bs,av_sell_price,reference,overhang,status,xu_cost,profit,loss,days_start,overhang_fac
<date>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<lgl>,<lgl>,<dbl>,<fct>
2005-12-29,47673.58,PETKM,7.95,7.936040,7.90,8.00,814312,6462412,21800,⋯,7.95,NaN,7.936040,-0.001757573,0,47673.58,FALSE,TRUE,0,0.6
2005-12-30,47689.19,PETKM,7.85,7.867242,7.80,7.95,945145,7435684,21800,⋯,7.95,NaN,7.867242,-0.010464411,0,47689.19,FALSE,TRUE,1,0.6
2005-03-21,29389.56,DENIZ,3.46,3.499026,3.40,3.66,676350,2366566,63900,⋯,3.60,NaN,3.499026,-0.028449225,0,29389.56,FALSE,TRUE,0,0.5
2005-03-22,28481.76,DENIZ,3.46,3.406361,3.34,3.48,1439584,4903743,63900,⋯,3.60,NaN,3.406361,-0.055289246,0,28481.76,FALSE,TRUE,1,0.4
2005-03-23,29215.08,DENIZ,3.22,3.266046,3.18,3.44,2245445,7333726,63900,⋯,3.60,NaN,3.266046,-0.097353810,0,29215.08,FALSE,TRUE,2,0.3
2005-03-24,29720.99,DENIZ,3.36,3.312746,3.20,3.38,2217923,7347415,63900,⋯,3.60,NaN,3.312746,-0.083156413,0,29720.99,FALSE,TRUE,3,0.4
2005-03-25,29047.00,DENIZ,3.40,3.370108,3.34,3.42,752149,2534824,63900,⋯,3.60,NaN,3.370108,-0.065988929,0,29047.00,FALSE,TRUE,4,0.4
2005-03-28,28630.19,DENIZ,3.28,3.312311,3.26,3.40,512954,1699063,63900,⋯,3.60,NaN,3.312311,-0.083287852,0,28630.19,FALSE,TRUE,5,0.4
2005-03-29,28773.84,DENIZ,3.28,3.281391,3.22,3.32,652749,2141925,63900,⋯,3.60,NaN,3.281391,-0.092666387,0,28773.84,FALSE,TRUE,6,0.3


In [121]:
pos_and_historic2[,levels(overhang_fac)]

[1] "0.1" "0.2" "0.3" "0.4" "0.5" "0.6" "0.7" "0.8" "0.9" "1"

In [122]:
dummies <- dummy_cols(pos_and_historic2, select_columns = "overhang_fac",
           remove_first_dummy = TRUE,
           ignore_na = T,
           remove_selected_columns = TRUE)

In [123]:
setDT(dummies)

In [124]:
dummies

date,value,MENKUL_KODU,KAPANIS,AOF,MIN,MAX,ISLEM_MIKTARI,ISLEM_HACMI,pos_no2,⋯,days_start,overhang_fac_0.2,overhang_fac_0.3,overhang_fac_0.4,overhang_fac_0.5,overhang_fac_0.6,overhang_fac_0.7,overhang_fac_0.8,overhang_fac_0.9,overhang_fac_1
<date>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2005-12-29,47673.58,PETKM,7.95,7.936040,7.90,8.00,814312,6462412,21800,⋯,0,0,0,0,0,1,0,0,0,0
2005-12-30,47689.19,PETKM,7.85,7.867242,7.80,7.95,945145,7435684,21800,⋯,1,0,0,0,0,1,0,0,0,0
2005-03-21,29389.56,DENIZ,3.46,3.499026,3.40,3.66,676350,2366566,63900,⋯,0,0,0,0,1,0,0,0,0,0
2005-03-22,28481.76,DENIZ,3.46,3.406361,3.34,3.48,1439584,4903743,63900,⋯,1,0,0,1,0,0,0,0,0,0
2005-03-23,29215.08,DENIZ,3.22,3.266046,3.18,3.44,2245445,7333726,63900,⋯,2,0,1,0,0,0,0,0,0,0
2005-03-24,29720.99,DENIZ,3.36,3.312746,3.20,3.38,2217923,7347415,63900,⋯,3,0,0,1,0,0,0,0,0,0
2005-03-25,29047.00,DENIZ,3.40,3.370108,3.34,3.42,752149,2534824,63900,⋯,4,0,0,1,0,0,0,0,0,0
2005-03-28,28630.19,DENIZ,3.28,3.312311,3.26,3.40,512954,1699063,63900,⋯,5,0,0,1,0,0,0,0,0,0
2005-03-29,28773.84,DENIZ,3.28,3.281391,3.22,3.32,652749,2141925,63900,⋯,6,0,1,0,0,0,0,0,0,0


In [125]:
formula1 <- paste(grep("overhang_fac.*\\d+$", names(dummies), value = T) %>% sort, collapse = " + ")
formula1

[1] "overhang_fac_0.2 + overhang_fac_0.3 + overhang_fac_0.4 + overhang_fac_0.5 + overhang_fac_0.6 + overhang_fac_0.7 + overhang_fac_0.8 + overhang_fac_0.9 + overhang_fac_1"

In [126]:
cox_cp3 <- coxph(sprintf("surv_cp1 ~ %s", formula1) %>% as.formula,
                 , id = pos_no2, cluster = pos_no2, data = dummies)

In [128]:
cox_cp3 %>% summary

Call:
coxph(formula = sprintf("surv_cp1 ~ %s", formula1) %>% as.formula, 
    data = dummies, id = pos_no2, cluster = pos_no2)

  n= 6573807, number of events= 147964 
   (27338 observations deleted due to missingness)

                    coef exp(coef) se(coef) robust se      z Pr(>|z|)    
overhang_fac_0.2 0.02448   1.02478  0.02138   0.02134  1.147  0.25119    
overhang_fac_0.3 0.03343   1.03400  0.02039   0.02037  1.641  0.10077    
overhang_fac_0.4 0.07395   1.07676  0.01956   0.01959  3.775  0.00016 ***
overhang_fac_0.5 0.09620   1.10098  0.01908   0.01922  5.006 5.57e-07 ***
overhang_fac_0.6 0.24194   1.27371  0.01875   0.01898 12.746  < 2e-16 ***
overhang_fac_0.7 0.53284   1.70376  0.01847   0.01879 28.352  < 2e-16 ***
overhang_fac_0.8 0.90399   2.46944  0.01821   0.01856 48.697  < 2e-16 ***
overhang_fac_0.9 0.89140   2.43853  0.01855   0.01891 47.142  < 2e-16 ***
overhang_fac_1   0.71097   2.03597  0.02034   0.02106 33.757  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0

In [129]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,8946647,477.9,14533755,776.2,22708991,1212.8
Vcells,338795805,2584.9,1132746792,8642.2,1415933490,10802.8


In [130]:
save.image("03_survival-01.RData")